# Bug in kripton MC (production-1)

### Description of error

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math

In [ ]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits

In [ ]:
import invisible_cities.sierpe.blr as blr

In [ ]:
import invisible_cities.core.mpl_functions as mpl

In [ ]:
import invisible_cities.core.peak_functions_c as cpf

### Test data

A MCRD file and a RWF file with one event resulting in an error has been produced and added to the repository

In [ ]:
os.environ['IC_NOTEBOOK_DIR'] 

In [ ]:
fmcrd = os.environ['IC_NOTEBOOK_DIR'] + '/files/kr_production1_bug_MCRD.h5'
frwf  = os.environ['IC_NOTEBOOK_DIR'] + '/files/kr_production1_bug_RWF.h5'

In [ ]:
h5mcrd = tb.open_file(fmcrd,'r+')
h5rwf  = tb.open_file(frwf,'r+')

### pmtrwf and sipmrwf vectors

In [ ]:

pmtrwf = h5rwf.root.RD.pmtrwf
pmtblr = h5rwf.root.RD.pmtblr
sipmrwf = h5rwf.root.RD.sipmrwf
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT()
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM(0)
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

In [ ]:
DataPMT

In [ ]:
DataSiPM.head()

### Step 1: Deconvolution (from RWF to CWF)

In [ ]:
event = 0
BLR = pmtblr[event]
RWF = pmtrwf[event]

In [ ]:
mpl.plot_waveforms(BLR)

In [ ]:
mpl.plot_waveforms(RWF)

In [ ]:
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr, n_baseline=28000, thr_trigger=5)
t1 = time()        
dt = t1 - t0

print("run over {} events  in {} s".format(event,dt))
mpl.plot_waveforms(CWF)

In [ ]:
mpl.plot_blr_cwf(BLR, CWF, zoom=True, window_size=600)

### Step 2: calibrated PMT sum

In [ ]:
event=0
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=200, thr_MAU=5) 
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
tstep = 25
signal_t = np.arange(0., PMTWL * tstep, tstep)

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=0, signal_end=1200, 
                ymax = 200, 
                t_units='mus', units="pes")

In [ ]:
np.sum(csum)

#### A protection is needed here, requesting the sum of the PMTs to be larger than a threshold!

In [ ]:
np.sum(wfzs_ene)

#### An empty waveform... looking for trouble!

### Step 4: Find S12 

S1 and S2 signals are found from the ZS waveform

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=5) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=1*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=8, lmax=20, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=100, lmax=1000000, stride=40,
                 rebin=True, rebin_stride=40)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

#### of course it fails! wfzs_ene is empty!

## Is the problem in DIOMIRA?

In [ ]:
h5mcrd = tb.open_file(fmcrd,'r+')

### diomira raw data

In [ ]:
pmtrd = h5mcrd.root.pmtrd
mpl.plot_waveforms(pmtrd[0])

#### The raw data is already corrupted (all PMTs equal to zero). The way to solve the problem is to add a protection to simulate_pmt_response.

In fact the protection can be very hard. One should not accept events unless there is signal (even if small) in all the PMTs, since there is no physical case for PMTs not having signal (S2 is large even for small energy depositions), and thus any event with a PMT totally empty is likely to be corrupted.